<a href="https://colab.research.google.com/github/DevCielo/neural-networks-from-scratch/blob/main/Neural_Networks_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bulding CNN's with tensorflow and from scratch**

# LeNet-5 with tensorflow


In [ ]:
import tensorflow as tf

def LeNet5():
  model = tf.keras.Sequential([
      # Convolution Layer
      tf.keras.layers.Conv2D(6, (5, 5), activation='tanh', strides=(1, 1), padding='valid', name='conv0'),
      # Average Pooling Layer
      tf.keras.layers.AveragePooling2D((2,2), strides=(2,2), name='avg_pool0'),
      # Convolution Layer
      tf.keras.layers.Conv2D(16, (5, 5), strides=(1,1), padding='valid', activation='tanh', name='conv1'),
      # Average Pooling Layer
      tf.keras.layers.AveragePooling2D((2,2), strides=(2,2), name='avg_pool1'),
      # Convolution Layer
      tf.keras.layers.Conv2D(120, (5, 5), strides=(1, 1), padding='valid', activation='tanh', name='conv2'),
      # Flatten the output
      tf.keras.layers.Flatten(),
      # Fully Connected Layer
      tf.keras.layers.Dense(84, activation='tanh', name='fc1'),
      tf.keras.layers.Dense(10, activation='softmax', name='output'),
  ])

  return model

# LeNet-5 from scratch

In [4]:
# Ensure you run this line
import numpy as np

In [5]:
def conv2d(input, filters, bias, stride=1, padding=0):
  # decompose the filters shape into the filter size (f) and the number of filters (n_f)
  (n_f, f, f) = filters.shape
  # decomposes the input shape into the input dimensions (in_dim) and the number of channels (n_c)
  (in_dim, in_dim, n_c) = input.shape

  # calculates the output dimensions using formula ((n+2p-f)/s + 1)
  # int to floor function
  out_dim = int((in_dim + 2*padding-f)/stride) + 1

  # sets the new output shape to be out_dim*out_dim*n_f
  output = np.zeros((out_dim, out_dim, n_f))

  # adds padding to the height and width but none to the number of channels
  input_padded = np.pad(input, ((padding, padding), (padding, padding), (0, 0)), mode='constant', constant_values = (0,0))

  # for every filter, performs the convolution by calculated the height/width start and ends and suming over their values to produce an output
  for i in range(n_f):
    for h in range(out_dim):
      for w in range(out_dim):
        h_start = h*stride
        h_end = h_start + f
        w_start = w*stride
        w_end = w_start + f
        output[h, w, i] = np.sum(input_padded[h_start:h_end, w_start:w_end, :] * filters[i]) + bias[i]

  return output


In [6]:
# Example usage of conv2d function (Equal to first step in LeNet-5)
input = np.random.randn(32, 32, 1)
filters = np.random.randn(6, 5, 5)
bias = np.random.randn(6)
output = conv2d(input, filters, bias, stride=1, padding=0)
print(output.shape)
# Expected Output: (28, 28, 6)

(28, 28, 6)


In [9]:
def average_pooling(input, size=2, stride=2):
  # decompose the input shape
  (in_dim, in_dim, n_c) = input.shape

  out_dim = int((in_dim - size)/ stride) + 1

  output = np.zeros((out_dim, out_dim, n_c))

  for c in range(n_c):
    for h in range(out_dim):
      for w in range(out_dim):
        h_start = h * stride
        h_end = h_start + size
        w_start = w * stride
        w_end = w_start + size
        output[h, w, c] = np.mean(input[h_start:h_end, w_start:w_end, c])

  return output


In [10]:
# Example usage after first convolution (Equal to second step in LeNet-5)
input = np.random.randn(28, 28, 6)
output = average_pooling(input, size=2, stride=2)
print(output.shape)
# Expected Output: (14, 14, 6)

(14, 14, 6)


In [13]:
def fully_connected(input, weights, bias):
  return np.dot(weights, input) + bias

In [15]:
# Example usage after flattening (Equal to second last step in LeNet-5)
input = np.random.randn(120)
weights = np.random.randn(84, 120)
bias = np.random.randn(84)
output = fully_connected(input, weights, bias)
print(output.shape)
# Expected Output: (84,)

(84,)


In [16]:
# Connecting developed functions into a class
class LeNet5:
    def __init__(self):
        self.conv1_filters = np.random.randn(6, 5, 5)
        self.conv1_bias = np.random.randn(6)

        self.conv2_filters = np.random.randn(16, 5, 5)
        self.conv2_bias = np.random.randn(16)

        self.conv3_filters = np.random.randn(120, 5, 5)
        self.conv3_bias = np.random.randn(120)

        self.fc1_weights = np.random.randn(84, 120)
        self.fc1_bias = np.random.randn(84)

        self.fc2_weights = np.random.randn(10, 84)
        self.fc2_bias = np.random.randn(10)

    # forward propagation
    def forward(self, x):
        # first layer
        x = conv2d(x, self.conv1_filters, self.conv1_bias, stride=1, padding=0)
        x = np.tanh(x)
        x = average_pooling(x, size=2, stride=2)

        # second layer
        x = conv2d(x, self.conv2_filters, self.conv2_bias, stride=1, padding=0)
        x = np.tanh(x)
        x = average_pooling(x, size=2, stride=2)

        # third layer
        x = conv2d(x, self.conv3_filters, self.conv3_bias, stride=1, padding=0)
        x = np.tanh(x)

        # flattening layer
        x = x.flatten()

        # fully connected layer
        x = fully_connected(x, self.fc1_weights, self.fc1_bias)
        x = np.tanh(x)

        # output layer
        x = fully_connected(x, self.fc2_weights, self.fc2_bias)
        x = softmax(x)

        return x

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=0)

# Create the model
lenet5 = LeNet5()

# Example input (32x32 grayscale image)
input = np.random.randn(32, 32, 1)
output = lenet5.forward(input)
print(output)

ValueError: operands could not be broadcast together with shapes (5,5,6) (5,5) 